In [17]:
!pip install transformers torch scikit-learn
!pip install sentence-transformers
!pip install nltk

In [2]:
import json
import re
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
import numpy as np

In [3]:
# Load the FAQs from the JSON file
with open('data/keelworks_info.json', 'r') as file:
    data = json.load(file)

In [4]:
faqs = data['questions_and_answers']


In [22]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Preprocess text (simple tokenization and lowercasing)
def preprocess_text(text):
    text = re.sub(r'\W+', ' ', text)  # Remove non-alphanumeric characters
    text = text.lower()  # Convert to lowercase
    return text

# Generate BERT embeddings
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.detach().numpy()

# Precompute embeddings for FAQ questions
faq_embeddings = [get_bert_embedding(preprocess_text(faq['question'])) for faq in faqs]
faq_embeddings = np.vstack(faq_embeddings)  # Ensure embeddings are in a 2D array

# Find the best matching answer
def get_best_answer(user_query, faqs, faq_embeddings):
    preprocessed_query = preprocess_text(user_query)
    query_embedding = get_bert_embedding(preprocessed_query).reshape(1, -1)

    similarities = cosine_similarity(query_embedding, faq_embeddings)
    best_match_index = similarities.argmax()

    return faqs[best_match_index]['answer']


In [23]:

# Command-Line Interface
def chatbot():
    print("Welcome to the KeelWorks Chatbot! Ask me anything about KeelWorks.")
    while True:
        user_query = input("\nYou: ")
        if user_query.lower() in ['exit', 'quit', 'bye']:
            print("Goodbye!")
            break
        answer = get_best_answer(user_query, faqs, faq_embeddings)
        print(f"Bot: {answer}")

if __name__ == '__main__':
    chatbot()

Welcome to the KeelWorks Chatbot! Ask me anything about KeelWorks.

You: what is the program called that assists new graduates
Bot: We emphasize critical thinking, self-identity, problem-solving, communication, assertiveness, teambuilding, goal determination, and goal management. We believe that those with these competencies will have all other competencies within reach.

You: exit
Goodbye!
